In [3]:
import newspaper
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd


count = 0
big_dataframe = pd.DataFrame({
    "Article Title": [],
    "Publish Date": [],
    "Clean Link": [],
    "Article URL": [],
    "Sentence Count": [],
    "Sum of Sentence Polarities": [],
    "Average Sentence Polarity": [],
    "Whole Article Polarity": []
})
for sheet in pd.ExcelFile('article_data_fixed.xlsx').sheet_names:
    data = pd.read_excel('article_data_fixed.xlsx', sheet_name=sheet)
    article_titles = []
    publish_dates = []
    clean_links = []
    article_urls = []
    sentence_counts = []
    sum_compound_polarities = []
    average_compound_polarities = []
    whole_commpound_polarities = []
    for i in range(len(data)):
        try:
            # Loads article into newspaper object via URL
            article_titles.append(data.loc[i, "title"])
            publish_dates.append(data.loc[i, "published_date"])
            clean_links.append(data.loc[i, "clean_url"])
            article_urls.append(data.loc[i, "link"])

            url = data.loc[i, "link"]
            article = newspaper.Article(url)
            article.download()
            article.parse()

            # Defines stopwords (common words such as pronouns and articles that can falsely influence sentiment)
            stopwords = nltk.corpus.stopwords.words("english")

            # Loads article text and tokenizes it into sentences, instantiates sentiment analysis object
            test_par = article.text
            vader = SentimentIntensityAnalyzer()
            test_sentences = sent_tokenize(test_par)
            polarity_scores = []
            sentences_stopped = []

            # Removes stopwords via tokenization and then converts back to string
            for sentence in test_sentences:
                sentence = word_tokenize(sentence)
                new_sentence = [w for w in sentence if w.lower() not in stopwords]
                #print(new_sentence)
                sentence_string = ""
                for word in new_sentence:
                    sentence_string = sentence_string + word + " "
                #print(sentence_string)
                sentences_stopped.append(sentence_string)
                #print(sentence_string)
                polarity_scores.append(vader.polarity_scores(sentence_string))
                #print(vader.polarity_scores(sentence_string))


            # Computes total and average polarity score
            total_polarity = 0
            for score in polarity_scores:
                total_polarity = total_polarity + score['compound']
            #print(total_polarity)
            average_polarity = total_polarity / len(polarity_scores)
            #print("This article contains " + str(len(polarity_scores)) + " sentences.")
            sentence_counts.append(len(polarity_scores))
            #print("The total compound polarity score for this article is: " + str(total_polarity) + ".")
            sum_compound_polarities.append(total_polarity)
            #print("The average compound polarity score for this article is: " + str(average_polarity) + ".")
            average_compound_polarities.append(average_polarity)
            #print("The compound polarity for this whole article is: " + str(vader.polarity_scores(test_par)['compound']) + ".")
            whole_commpound_polarities.append(vader.polarity_scores(test_par)['compound'])

            small_dataframe = pd.DataFrame({
                "Article Title": article_titles,
                "Publish Date": publish_dates,
                "Clean Link": clean_links,
                "Article URL": article_urls,
                "Sentence Count": sentence_counts,
                "Sum of Sentence Polarities": sum_compound_polarities,
                "Average Sentence Polarity": average_compound_polarities,
                "Whole Article Polarity": whole_commpound_polarities
            })

            big_dataframe = pd.concat([big_dataframe, small_dataframe])
            count = count + 1
            print(count)
        except:
            print(data.loc[i, "link"])
            continue

with pd.ExcelWriter("polarity_scores.xlsx", mode='a', if_sheet_exists='replace') as writer:
    big_dataframe.to_excel(writer)


AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)